<a href="https://colab.research.google.com/github/arminsoltan/mohaymen/blob/main/Phase05/CNN_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [758]:
import pandas as pd

train_data = pd.read_csv("/content/drive/MyDrive/Mohaymen/Phase05/snappfood/train_data_preprocessing.csv")
test_data = pd.read_csv("/content/drive/MyDrive/Mohaymen/Phase05/snappfood/test_data_preprocessing.csv")
valid_data = pd.read_csv("/content/drive/MyDrive/Mohaymen/Phase05/snappfood/valid_data_preprocessing.csv")

In [759]:
train_data.head()

,Unnamed: 0,comment,label_id,preprocessing
0,0,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,1,حیف وقت نوشت نویس سرویس دهیتون افتضاح
1,1,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,0,قرار بود باش ساعته برسه نیم ساعت موقع دید بین ...
2,2,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,1,قیمت مدل کیفیت سازگاری نداره ظاهر فریبنده داره...
3,3,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,0,عالللی بود باش درست اندازه کیفیت امیداورم کیفی...
4,4,شیرینی وانیلی فقط یک مدل بود.,0,شیرینی نیل مدل بود باش


In [760]:
import torch
from torchtext.legacy import data
if torch.cuda.is_available():
  TRAINING_ON_GPU = True
else:
  TRAINING_ON_GPU = False
SEED = 2021
torch.manual_seed(SEED)
TEXT = data.Field(tokenize='spacy', batch_first=True)
LABEL = data.LabelField(dtype=torch.float, batch_first=True)
fields = [(None, None), (None, None), ('label_id', LABEL), ('preprocessing', TEXT)]

In [761]:
train_data = data.TabularDataset(path="/content/drive/MyDrive/Mohaymen/Phase05/snappfood/train_data_preprocessing.csv", format="csv", fields=fields, skip_header=True)
test_data = data.TabularDataset(path="/content/drive/MyDrive/Mohaymen/Phase05/snappfood/test_data_preprocessing.csv", format="csv", fields=fields, skip_header=True)
valid_data = data.TabularDataset(path="/content/drive/MyDrive/Mohaymen/Phase05/snappfood/valid_data_preprocessing.csv", format="csv", fields=fields, skip_header=True)

In [762]:
# import torch.nn as nn
# import torch.nn.functional as F

# class CNN(nn.Module):
#   def __init__(self, vocab_size, embed_size, n_filters, filter_sizes, pool_size, hidden_size, num_classes, dropout):
#     super().__init__()
#     self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embed_size)
#     self.convs = nn.ModuleList([nn.Conv1d(in_channels=1, out_channels=n_filters, kernel_size=(fs, embed_size)) for fs in filter_sizes])

#     self.max_pool1 = nn.MaxPool1d(pool_size)
#     self.relu = nn.ReLU()
#     self.dropout = nn.Dropout(dropout)
#     self.fc1 = nn.Linear(13 * n_filters, hidden_size, bias=True)
#     self.fc2 = nn.Linear(hidden_size, num_classes, bias=True)

#   def forward(self, text):
#     # print("text length \n", text_length)
#     # [batch, ]
#     embedded = self.embedding(text)
#     print("embedded \n", embedded.shape)
#     embedded = F.pad(embedded, (0, 0, 2, 2), "constant", 0)
#     print("embedded \n", embedded.shape)
#     embedded = embedded.unsqueeze(1)
#     print("embedded \n", embedded.shape)
#     convolution = [conv(embedded) for conv in self.convs]
#     print("\n convolution[0] \n", convolution[0].shape, "\n")
#     print("\n convolution[1] \n", convolution[1].shape, "\n")
#     # print(convolution)
#     max1 = self.max_pool1(convolution[0].squeeze())
#     print("max1 \n", max1.shape)
#     max2 = self.max_pool1(convolution[1].squeeze())
#     print("max2 \n", max2.shape)
#     cat = torch.cat((max1, max2), dim=2)
#     print("cat \n", cat.shape)
#     x = cat.view(cat.shape[0], -1)
#     print("concat max1, max2 \n", x.shape)
#     x = x.squeeze(1)
#     print("x squeeze\n", x.shape)
#     x = self.relu(x)
#     print("relu(x)")
#     print(x.shape)
#     print(x)
#     print(type(x))
#     x = self.fc1(x)
#     print(x)
#     x = self.dropout(x)
#     x = self.fc2(x)
#     return x

In [763]:
import torch.nn as nn
import torch.nn.functional as F
class CNN(nn.Module):
  def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim
               , dropout, pad_idx):
    super(CNN, self).__init__()
    
    # we consider embedding layer as a lookup table that assigns each word a vector
    self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
    
    # we define convolutional layers in ModuleList
    self.convs = nn.ModuleList([nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(fs, embedding_dim)) 
                                         for fs in filter_sizes])

    self.fc = nn.Linear(len(filter_sizes)*n_filters, output_dim)

    self.dropout = nn.Dropout(dropout)
    self.sigmoid = nn.Sigmoid()

  def forward(self, text):
    
    # text = [batch_size, sent length]
    embedded = self.embedding(text)

    # because min len text ==1, we should add 4 to it dimention, because our max kernel size is 5
    # padding
    # embedded = [batch_size, sent length, emb dim]
    
    embedded = F.pad(embedded, (0,0,2,2), "constant", 0)
    # embedded = [batch_size, sent length + 4, emb dim]

    embedded = embedded.unsqueeze(1)
    # embedded = [batch_size, 1, sent length + 4, emb dim]

    conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
    # conved_n = [batch_size, n_filters, sent length + 4 -filter_sizes[n] + 1]
    # print("conved\n", conved[0].shape, "\n", len(conved))

    pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
    # pooled_n = [batch_size, n_filters]
    # print("pooled\n", pooled[0].shape, "\n", len(pooled))

    cat = self.dropout(torch.cat(pooled, dim=1))
    # print("cat\n", cat.shape)
    # cat = [batch_size, n_filters * len(filter_sizes)]
    
    dense_output = self.fc(cat)

    # final activation funciton 
    output = self.sigmoid(dense_output)

    return output

In [764]:
lr = 0.1
batch_size = 50
dropout_keep_prob = 0.5
embedding_size = 100
max_document_length = 100
max_size = 5000
num_classes = 2
hidden_size = 128
pool_size = 2
n_filters = 100
filter_sizes = [3, 4, 5]
num_epochs = 5


In [765]:
TEXT.build_vocab(train_data, max_size=max_size)
LABEL.build_vocab(train_data)
vocab_size = len(TEXT.vocab)
pad_idx = TEXT.vocab.stoi[TEXT.pad_token]

In [766]:
train_iterator = data.BucketIterator(train_data, batch_size, sort_key= lambda x: len(x.preprocessing), train=True, shuffle=True, sort_within_batch=True)
valid_iterator = data.BucketIterator(valid_data, batch_size, sort_key= lambda x: len(x.preprocessing), train=False, shuffle=False, sort_within_batch=True)
test_iterator = data.BucketIterator(test_data, batch_size, sort_key= lambda x: len(x.preprocessing), train=False, shuffle=False, sort_within_batch=True)

In [767]:
def accuracy(y_pred, y_act):
    rounded_preds = torch.round(y_pred)
    correct = (rounded_preds == y_act).float()
    acc = correct.sum() / len(correct)
    acc = torch.round(acc * 100)
    return acc

In [768]:
from tqdm import tqdm
import torch.nn as nn
import torch

# net = CNN(vocab_size, embedding_size, n_filters, filter_sizes, pool_size, hidden_size, num_classes, dropout_keep_prob)
net = CNN(vocab_size, embedding_size, n_filters, filter_sizes, 1, dropout_keep_prob, pad_idx)
if TRAINING_ON_GPU:
  net.cuda()
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.5)
acc = 0

# for i, batch in enumerate(train_iterator):
#   text = batch.preprocessing
#   print(text.shape)
# return 
v = torch.tensor([1.0 for i in range(50)])
t = tqdm(range(1, 1 + num_epochs))
for i in t:
  t.set_description("epoch: {}".format(i))
  t.refresh()
  net.train()
  acc = 0
  for j, batch in enumerate(train_iterator):
    optimizer.zero_grad()
    # net.zero_grad()
    if TRAINING_ON_GPU:
      input = batch.preprocessing.cuda()
      label = batch.label_id.cuda()
    else:
      input = batch.preprocessing
      label = batch.label_id
    
    # print(input.shape, len(input))
    # print(input)
    
    output = net(input).squeeze()    
  # if (j==1 or j==2):
    # print(input)
    correct = (v == torch.round(output)).float()
    threshold = correct.sum()
    if (threshold != 50):
      print("epoch: {} va batch: {}".format(i, j))
      print(output)
      print(torch.round(output))
      print(label)
    # output = torch.tensor(output.argmax(dim=1), dtype=torch.float)
    # print("output\n", output)
    # print("output squeeze \n", output.squeeze(), output.shape)
    # print("label squeeze \n", label.squeeze().float(), label.shape)
    loss = criterion(output, label)
    # loss.requires_grad= True
    # loss.requires_grad = False
    # loss.grad_fn = False
    # loss.grad = False
    loss.backward()
    optimizer.step()

  scheduler.step()
  net.eval()
  acc = 0
  for j, batch in enumerate(valid_iterator):
    # optimizer.zero_grad()
    with torch.no_grad():
      if TRAINING_ON_GPU:
        input = batch.preprocessing.cuda()
        label = batch.label_id.cuda()
      else:
        input = batch.preprocessing
        label = batch.label_id
      output = net(input).squeeze()
      acc += accuracy(output, label)
  print(acc / len(valid_iterator))
  t.set_description("accuracy: {}".format(acc / len(valid_iterator)))

epoch: 1:   0%|          | 0/5 [00:00<?, ?it/s]

epoch: 1 va batch: 0
tensor([0.2730, 0.5675, 0.3357, 0.7023, 0.6098, 0.3845, 0.3413, 0.2821, 0.4841,
        0.5922, 0.6667, 0.2635, 0.3810, 0.4429, 0.4170, 0.2493, 0.5487, 0.4904,
        0.6613, 0.4235, 0.6524, 0.6582, 0.3957, 0.3933, 0.3915, 0.6758, 0.5318,
        0.2983, 0.5822, 0.5360, 0.6798, 0.6954, 0.5519, 0.5722, 0.4119, 0.4897,
        0.4900, 0.3426, 0.6119, 0.7483, 0.4570, 0.4419, 0.2947, 0.6871, 0.6406,
        0.2208, 0.3982, 0.3988, 0.4737, 0.5268], grad_fn=<SqueezeBackward0>)
tensor([0., 1., 0., 1., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0.,
        1., 0., 1., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 0.,
        0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1.],
       grad_fn=<RoundBackward>)
tensor([0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 1., 1., 1., 1., 0.,
        0., 1., 0., 0., 1., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1., 0., 0., 1.,
        0., 0., 0., 1., 0., 0., 0., 1., 1., 1., 0., 0., 1., 0.])
epoch: 1 va batch: 1

epoch: 2:  20%|██        | 1/5 [00:55<03:40, 55.14s/it]

tensor(50.)


epoch: 3:  40%|████      | 2/5 [01:51<02:48, 56.02s/it]

tensor(50.)


epoch: 3:  40%|████      | 2/5 [02:16<03:24, 68.00s/it]


KeyboardInterrupt: ignored

In [ ]:
net.eval()
acc = 0
for i, batch in enumerate(test_iterator):
  with torch.no_grad():
    # optimizer.zero_grad()
    if TRAINING_ON_GPU:
      input = batch.preprocessing.cuda()
      label = batch.label_id.cuda()
    else:
      input = batch.preprocessing
      label = batch.label_id
    output = net(input).squeeze()
    if (i % 10 == 0):
      print(input.shape)
      print(output)
    acc += accuracy(output, label)
print(acc / len(test_iterator))

In [ ]:
v = torch.tensor([1.0 for i in range(50)])
print(v)
r = torch.round(v)
print(v, "\n", r)